In [ ]:
pip install langchain

In [2]:
from langchain.output_parsers import StructuredOutputParser,ResponseSchema
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import json
import pandas as pd

In [3]:
openai_api_key = '...'

In [4]:
# Temp =  0 para que la información limpia no se ponga muy creativa y aleatoria
chat_model = ChatOpenAI(temperature=0,openai_api_key=openai_api_key,max_tokens=1000)

In [6]:
# Como queremos nuestra respuesta, piénsalo como un prompt estructurado pero para lo que queremos recibir al final
response_schemas = [
    ResponseSchema(name="movie", description="Este es el input del usuario para que busque el género de la película"),
    ResponseSchema(name="standardized_genre", description="Este es el género de película a la que crees que el input del usuario es más parecido"),
    ResponseSchema(name="match_score", description="Un score entre 0-100 de que tan cerca está el user input y tu match")
]

# Como te gustaría formatear tu output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [7]:
# vamos a ver el prompt template que creamos para formatear la salida
format_instructions = output_parser.get_format_instructions()
print(output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"movie": string  // Este es el input del usuario para que busque el género de la película
	"standardized_genre": string  // Este es el género de película a la que crees que el input del usuario es más parecido
	"match_score": string  // Un score entre 0-100 de que tan cerca está el user input y tu match
}
```


In [8]:
template = """
Se te proporcionará una serie de películas ingresados por un usuario.
Encuentra la mejor coincidencia correspondiente en la lista de géneros estandarizados.
La coincidencia más cercana será aquella con el significado semántico más próximo, no solo similitud de cadenas de texto.

{format_instructions}

Envuelve tu salida final con corchetes abiertos y cerrados (una lista de objetos JSON).

PELÍCULAS INGRESADOS INPUT:
{user_movies}

GÉNEROS ESTANDARIZADOS DE PELÍCULAS:
{standarized_movie_genres}

"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)
    ],
    input_variables=["user_movies","standarized_movie_genres"],
    partial_variables={"format_instructions":format_instructions}
)

In [9]:
# Damos la lista de nombres estandarizados
df = pd.read_csv("./data/Genre_Movies.csv")
standarized_movie_genres = ", ".join(df['Genero'].values)
standarized_movie_genres

'Acción, Aventura, Comedia, Drama, Terror, Suspenso, Ciencia Ficción, Fantasía, Animación, Documental, Romance, Musical, Misterio, Crimen, Bélico, Histórico, Biográfico, Deporte, Western, Familiar'

In [10]:
# El input de tu usuario

user_input = "Forest Gump (la de correr), El Senor de los Anillos: La Comunidad del Aniyo, Buscando a Nemo (la del pes), Insepcion, El Padrino parte dos, Jurasic Park (la primera), Volver al futuro tres, La guerra de las Galaxias: Una nueva esperansa, Matriks, Avatar (la de los aliens azules), Gladiador, En busca de la felisidad (Will Smith), Interestelar, Harry Poter y la piedra filosofal, El código Da Binci, Piratas del Caribe: La maldision de la Perla Negra, Rápido y furioso: Reto Tokyo, El Diario de Bridget Jones, El Gran Gatsby (con DiCaprio), 500 días con ella"
_input = prompt.format_prompt(user_movies=user_input,standarized_movie_genres=standarized_movie_genres)

print(f"Hay {len(_input.messages)} inputs")
print(f"Type: {type(_input.messages[0])}")
print("-------------")
print(_input.messages[0].content)

Hay 1 inputs
Type: <class 'langchain.schema.HumanMessage'>
-------------

Se te proporcionará una serie de películas ingresados por un usuario.
Encuentra la mejor coincidencia correspondiente en la lista de géneros estandarizados.
La coincidencia más cercana será aquella con el significado semántico más próximo, no solo similitud de cadenas de texto.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"movie": string  // Este es el input del usuario para que busque el género de la película
	"standardized_genre": string  // Este es el género de película a la que crees que el input del usuario es más parecido
	"match_score": string  // Un score entre 0-100 de que tan cerca está el user input y tu match
}
```

Envuelve tu salida final con corchetes abiertos y cerrados (una lista de objetos JSON).

PELÍCULAS INGRESADOS INPUT:
Forest Gump (la de correr), El Senor de los Anillos: La Comunidad del

In [15]:
output = chat_model(_input.to_messages())

In [16]:
print(type(output))
print(output.content)

<class 'langchain.schema.AIMessage'>
```json
[
	{
		"movie": "Forest Gump (la de correr)",
		"standardized_genre": "Drama",
		"match_score": "100"
	},
	{
		"movie": "El Senor de los Anillos: La Comunidad del Aniyo",
		"standardized_genre": "Fantasía",
		"match_score": "100"
	},
	{
		"movie": "Buscando a Nemo (la del pes)",
		"standardized_genre": "Animación",
		"match_score": "100"
	},
	{
		"movie": "Insepcion",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"movie": "El Padrino parte dos",
		"standardized_genre": "Crimen",
		"match_score": "100"
	},
	{
		"movie": "Jurasic Park (la primera)",
		"standardized_genre": "Aventura",
		"match_score": "100"
	},
	{
		"movie": "Volver al futuro tres",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"movie": "La guerra de las Galaxias: Una nueva esperansa",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"movie": "Matriks",
		"standardized_genre": "Ciencia Ficción"

In [13]:
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

In [14]:
print(output.content)

```json
[
	{
		"movie": "Forest Gump (la de correr)",
		"standardized_genre": "Drama",
		"match_score": "100"
	},
	{
		"movie": "El Senor de los Anillos: La Comunidad del Aniyo",
		"standardized_genre": "Fantasía",
		"match_score": "100"
	},
	{
		"movie": "Buscando a Nemo (la del pes)",
		"standardized_genre": "Animación",
		"match_score": "100"
	},
	{
		"movie": "Insepcion",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"movie": "El Padrino parte dos",
		"standardized_genre": "Crimen",
		"match_score": "100"
	},
	{
		"movie": "Jurasic Park (la primera)",
		"standardized_genre": "Aventura",
		"match_score": "100"
	},
	{
		"movie": "Volver al futuro tres",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"movie": "La guerra de las Galaxias: Una nueva esperansa",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"movie": "Matriks",
		"standardized_genre": "Ciencia Ficción",
		"match_score": "100"
	},
	{
		"mo